In [ ]:
from dep_tools.namers import DepItemPath
import folium
from pystac import Item
from odc.stac import load, configure_s3_access

In [ ]:
version_1 = "0.3.0"
version_2 = "0.3.1"

sensor = "s2"
dataset_id = "geomad"
year = "2023"

bucket_url = "https://dep-public-staging.s3.us-west-2.amazonaws.com/"

In [ ]:
areas = (
    ("63,20", "Viti_Levu"),
    ("48,16", "Vanuatu"),
    ("89,16", "Rarotonga"),
    ("56,49", "Kiribati"),
    ("75,25", "Samoa"),
    ("28,32", "PNG")
)


configure_s3_access(cloud_defaults=True, aws_unsigned=True)

for item_id, name in areas:
    m = folium.Map(location=[0, 0], zoom_start=2)

    for version in [version_2, version_1]:
        zp = version == "0.3.1"
        item_path = DepItemPath(sensor, dataset_id, version, time=year, zero_pad_numbers=zp)
        item_url = bucket_url + item_path.stac_path(item_id=item_id)
        item = Item.from_file(item_url)
        data = load([item], bands=["red", "green", "blue"], chunks={})

        data.squeeze().odc.to_rgba(vmin=1000, vmax=4000).odc.add_to(m, name=f"v: {version}")

    m.fit_bounds(data.odc.map_bounds())
    folium.LayerControl().add_to(m)

    m.save(f"comparison_{name}.html")

In [ ]:
item_id = "032,030"
item_id = "078,025"

# https://stac.staging.digitalearthpacific.io/collections/dep_s2_geomad/items/dep_s2_geomad_032_029_2023
# https://stac.staging.digitalearthpacific.io/collections/dep_s2_geomad/items/dep_s2_geomad_032_030_2023

m = folium.Map(location=[0, 0], zoom_start=2)

for version in [version_2]:
    zp = version == "0.3.1"
    item_path = DepItemPath(sensor, dataset_id, version, time=year, zero_pad_numbers=zp)
    item_url = bucket_url + item_path.stac_path(item_id=item_id)
    item = Item.from_file(item_url)
    data = load([item], bands=["red", "green", "blue"], chunks={})

    data.squeeze().odc.to_rgba(vmin=1000, vmax=4000).odc.add_to(m, name=f"v: {version}")

m.fit_bounds(data.odc.map_bounds())
folium.LayerControl().add_to(m)

m